In [1]:
import PyPDF2 as pypdf
import reportlab
#paper sizes
import reportlab.lib.pagesizes as pagesize
import tempfile
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm

In [4]:
inputsig = pypdf.PdfFileReader('1-8.pdf')

In [2]:
import tempfile
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm

In [2]:
pts_to_mm = 2.8346456693
def pts_mm(size:tuple, mode=True):
    if mode: #pts to mm
        x = round(size[0]/pts_to_mm,2)
        y = round(size[1]/pts_to_mm,2)
        return (x,y)
    else:   #mm to pts
        x = round(size[0] * pts_to_mm, 2)
        y = round(size[1] * pts_to_mm, 2)
        return (x,y)

In [4]:
a4 = (210,297) #mm
print(pts_mm(a4, False))
print((a4[0]*mm,a4[1]*mm))

(595.28, 841.89)
(595.2755905511812, 841.8897637795277)


In [7]:
print(pts_mm((a4[0]*mm,a4[1]*mm)))

(210.0, 297.0)


In [5]:
print(mm)

2.834645669291339


In [3]:
fp = tempfile.TemporaryFile()
can = canvas.Canvas(fp, pagesize=(2*595.27, 2*841.89))

In [5]:
can.lines([((0,100, 100,100)), ((200,100, 400,100))])

In [32]:
can.drawString(100,100," Test Pages")

In [34]:
can.setStrokeColorRGB(0.2,0.5,0.3)
can.setFillColorRGB(1,0,1)
#an draw some lines
can.line(0,0,0,1.7*inch)
can.line(0,0,1*inch,0)
#an draw a rectangle
can.rect(0.2*inch,0.2*inch,1*inch,1.5*inch, fill=1)
#an make text go straight up
can.rotate(90)
#an change color
can.setFillColorRGB(0,0,0.77)
#an say hello (note after rotate the y coord needs to be negative!)
can.drawString(0.3*inch, -inch, "Hello World")

In [35]:
can.save()

In [36]:
twopdf = pypdf.PdfFileReader(fp)

In [37]:
genpdf = pypdf.PdfFileWriter()

In [47]:
inputpage = inputsig.getPage(1)
inputpage.add_transformation(pypdf.Transformation().rotate(90).translate(tx=10))

In [49]:
page = twopdf.getPage(0)
page.merge_page(inputpage)

In [50]:
genpdf.add_page(inputpage)
with open("./test.pdf", "wb") as f:
    genpdf.write(f)

In [ ]:
twopdf = pypdf.PdfFileReader(fp)

In [59]:
inputsig = pypdf.PdfFileReader('1-8.pdf')
inputpage = inputsig.getPage(0)
op = pypdf.Transformation().rotate(0).translate(ty=700, tx=500)
inputpage.addTransformation(op)
pagetwo = twopdf.getPage(0)
pagetwo.merge_page(inputpage)


genpdf = pypdf.PdfFileWriter()
genpdf.add_page(pagetwo)

with open("./copy_1-8.pdf", "wb") as f:
    genpdf.write(f)

In [69]:
import math,io
import PyPDF2 as pypdf
import reportlab
#paper sizes
import reportlab.lib.pagesizes as pagesize
import tempfile
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm

def convert(hex:str)->tuple:
    if "#" in hex:
        hex= hex.replace("#","")
    R, G, B  = int(hex[0:2],16), int(hex[2:4],16), int(hex[4:6],16)
    
    R = R/255
    G = G/255
    B = B/255

    K = 1- max(R,G,B)
    C = (1-R-K)/(1 - K)
    M = (1-G-K)/(1 - K)
    Y = (1-B-K)/(1 - K)
    return C, M, Y, K

def sig_layout(n:int) -> tuple:
        if type(n) != int or n<4 or n%4 !=0:
            raise ValueError(f"n:{n} must be a positive integer that multiple of 4.")

        if n%3 ==0:
            i = math.log2(n) - math.log2(3) -1
            return(3, int(2**i))
        else:
            i = int(math.log2(n/4))
            if i%2 :
                k = kp = int((i+1)/2)
            else:
                k = int(i/2)
                kp = k+1
            return (int(2**k), int(2**kp))

# e.g: generate_layout((,), (1,4), n=20, d=5, proof=True, proofcode="", trim=True, registration=True, cmyk=True)
def generate_layout(#All dimensions are written in pts unit
        pagesize:tuple, 
        pagenum:int,
        n:tuple, nd:int, d:int, 
        proof:bool, proofcode:str, 
        trim:bool, 
        registration:bool, 
        cmyk:bool
    ):

        #Signature composition
        ni = n[0]
        ns = n[1]

        n_block = int(pagenum/(ni * ns))

        
        #Paper Dimension
        arrange =sig_layout(ns)
        ny = arrange[0]
        nx = arrange[1]
        x = 2*nd + nx*pagesize[0] + (nx-1)*d    
        y = 2*nd + ny*pagesize[1] + (ny-1)*d

        

        #Signature proof
        if proof:
            proof_height = pagesize[1]/n_block
            proof_width = d
            cmyk_proof = convert(proofcode)
            proof_position = [nd+pagesize[0], nd+ny*pagesize[1] + (ny-1)*d-proof_height]
        #trim
        if trim:
            trim_l = nd*(1/2)
            #horizontal line
            x1 = nd/4
            x2 = nd + nx*pagesize[0] + (nx-1)*d +x1
            y1 = nd + ny*pagesize[1] + (ny-1)*d
            y2 = nd
            #vertical line
            x3 = nd
            x4 = x2 - x1
            y3 = nd/4
            y4 = y1 + y3

            trim_lines = [
                (x1,y1, x1 + trim_l, y1), # h, u l
                (x1,y2, x1 + trim_l, y2), # h, d l
                (x2,y1, x2 + trim_l, y1), # h, u r
                (x2,y2, x2 + trim_l, y2), # h, d r
                (x3,y4, x3, y4 + trim_l), # v, u l
                (x3,y3, x3, y3 + trim_l), # v, d l
                (x4,y4, x4, y4 + trim_l), # v, u r
                (x4,y3, x4, y3 + trim_l)  # v, d r
            ]
        if registration:
            reg_l = nd/2
            pass
        if cmyk:
            rec_l = nd/2
            rec_d = nd/8
            cmyk_position = [nd/4, y1-rec_l*2]

        tem_pdf_byte = io.BytesIO()

        layout = canvas.Canvas(tem_pdf_byte, pagesize = (x,y))

        for i in range(0, n_block):
            for j in range(0, ni):

                #fill basic layout components
                if proof and j==0 : # draw rectangle
                    layout.setLineWidth(0)
                    layout.setFillColorCMYK(cmyk_proof[0], cmyk_proof[1] ,cmyk_proof[2], cmyk_proof[3])
                    layout.rect(proof_position[0], proof_position[1], proof_width, proof_height, fill=1)

                    proof_position[1] = proof_position[1] - proof_height
                for k in range(0,2):  
                    if trim: # draw line
                        layout.setLineWidth(0.5*mm)
                        layout.lines(trim_lines)
                    if registration: # add image
                        pass
                    if cmyk: 
                        layout.setLineWidth(0)
                        layout.setFillColor(color_cyan)
                        layout.rect(cmyk_position[0], cmyk_position[1], rec_l, rec_l, fill=1)
                        cmyk_position[1] -=(rec_d + rec_l)
                        layout.setFillColor(color_magenta)
                        layout.rect(cmyk_position[0], cmyk_position[1], rec_l, rec_l, fill=1)
                        cmyk_position[1] -=(rec_d + rec_l)
                        layout.setFillColor(color_yellow)
                        layout.rect(cmyk_position[0], cmyk_position[1], rec_l, rec_l, fill=1)
                        cmyk_position[1] -=(rec_d + rec_l)
                        layout.setFillColor(color_black)
                        layout.rect(cmyk_position[0], cmyk_position[1], rec_l, rec_l, fill=1)
                        cmyk_position[1] = y1-rec_l*2


                    layout.showPage()

        #----------------------------
        layout.save()
        tem_pdf_byte.seek(0)
        tem_pdf  = pypdf.PdfReader(tem_pdf_byte)
        return tem_pdf, tem_pdf_byte

In [47]:
from reportlab.lib.colors import CMYKColor

color_black = CMYKColor(0, 0, 0, 1)
color_cyan = CMYKColor(1, 0, 0, 0)
color_magenta = CMYKColor(0, 1, 0, 0)
color_yellow = CMYKColor(0, 0, 1, 0)


In [28]:
print(color_cyan)

CMYKColor(1,0,0,0,alpha=1)


In [77]:
test, test_byte = generate_layout(
    (595.28, 841.89), 
    48,
    (1,12), 
    nd=20, 
    d=5, 
    proof=True, 
    proofcode="#ff5733", 
    trim=True, 
    registration=True, 
    cmyk=True
    )
testwrite = pypdf.PdfFileWriter()
for i in range(0, test.numPages):
    testwrite.add_page(test.getPage(i))

with open("layout.pdf", "wb") as f:
    testwrite.write(f)


In [40]:
test.numPages

4

In [76]:
sig_layout(24)

(3, 4)

In [15]:
if '#' in '#ff5733':
    print('#ff5733'.replace("#",""))

ff5733


In [20]:
(1,2)[0]

1